In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.naive_bayes import GaussianNB

In [ ]:
# load dataset
df = pd.read_csv('/data/raw-data-crop-recommendation.csv')

# split the dataframe into features (x) and labels (y)
x = df.drop(columns = ['label'])
y = df['label']

In [ ]:
# split the data to 90-10 where 90% is for training and testing while the remaining 10% is for unseen data
x_train_test, x_unseen, y_train_test, y_unseen = train_test_split(x, y, test_size = 0.10, random_state = 42)

# split the train_test to 80-20 where 80% is for training while the 20% is for testing
x_train, x_test, y_train, y_test = train_test_split(x_train_test, y_train_test, test_size = 0.20, random_state = 42)

# perform a ten-fold cross-validation
kf = KFold(n_splits = 10, shuffle = True, random_state = 42)

In [ ]:
# instantiate the gaussian naive bayes model
model = GaussianNB()

# train the model
model.fit(x_train, y_train)

# predict on the test set
y_pred = model.predict(x_test)

print(y_pred)